In [2]:
from bs4 import BeautifulSoup
import pymongo
import requests
import re
import sys

In [3]:
# Create mongodb connection
from pymongo import MongoClient
connection = MongoClient('localhost', 27017)
database = connection.housing_data

table = database.etuovi

#table.drop() to drop the collection 'housing_data'

cookie = {'org.springframework.mobile.device.site.CookieSitePreferenceRepository.SITE_PREFERENCE':'NORMAL','JSESSIONID':'183EE2F12C128B603790A0D5D1558022','evid_0006':'87ccc59d-4b80-489f-98cf-be92d6591fa6','evid_0006_ref':'false','adptseg_0006':'kv1001#a-kv1002#c-kv1003#d-kv1004#d-kv1005#c-kv1006#a:b-kv1008#c-kv1009#17-kv1010#b-kv1011#f-kv1012#c-kv1013#a-kv1016#1:11:13:2:7-kv1017#2-','adptset_0006':'1','evid_0006_set':'2','__utma':'82691985.304271103.1385626909.1385642039.1385658686.4','__utmb':'82691985.9.8.1385658723878','__utmc':'82691985','__utmz':'82691985.1385626909.1.1.utmcsr=kuluttaja.etuovi.com|utmccn=(referral)|utmcmd=referral|utmcct=/crometapp/product/realties/common/public/frontpage2/search_in_progress_to_vuokraovi.jsp','slbp_80':'2265258762.20480.0000','slbp_443':'2265258762.47873.0000'}

data_added = 0

In [6]:
start_page = 1
end_page = 3
Links = []
for page in range(start_page, end_page):
    print ('Parsing page:', str(page))
    html = requests.post('http://www.vuokraovi.com/vuokra-asunnot?page=' + str(page), cookies=cookie, data={'listorder':11})
    
    parsed_html = BeautifulSoup(html.text,'html.parser')                                                                                        
    #print(parsed_html.prettify())
    url_tag = parsed_html.find_all('a', {'class':'list-item-link'})
    for link in url_tag:
        address = link.get('href')
        if '/vuokra-asunto' in address:
            #print(address)
            Links.append(address)
#print (Links)

Parsing page: 1
Parsing page: 2


In [5]:
allApartments = []
apartment = dict()
for page in Links:
    #print(page)
    html = requests.post('http://www.vuokraovi.com' + page, cookies=cookie, data={'listorder':11})
    parsed_html = BeautifulSoup(html.text,'html.parser') 
    #print(parsed_html)
    test_tagth = parsed_html.find_all('th')
    test_tagtd = parsed_html.find_all('td')
    apartment['postal_code'] = test_tagtd[0].find('span', {'itemprop':'addressLocality'}).get_text(strip = True)[:5]
  
    
    for i in range(0,len(test_tagth)):
        if 'Vakuus:' in test_tagth[i].get_text():
            apartment['deposit'] = test_tagtd[i].get_text(strip = True)
            
        if 'Vuokra:' in test_tagth[i].get_text():
            apartment['rent'] = test_tagtd[i].get_text(strip = True) 
            
        if 'Tyyppi:' in test_tagth[i].get_text():
            apartment['type'] = test_tagtd[i].get_text(strip = True)
            
        if 'Huoneiden lukumäärä' in test_tagth[i].get_text():
            apartment['number_of_rooms'] = test_tagtd[i].get_text(strip = True)
            
        if 'Rakennusvuosi:' in test_tagth[i].get_text():
            apartment['built_year'] = test_tagtd[i].get_text(strip = True)
            
        if 'Yleiskunto:' in test_tagth[i].get_text():
            apartment['condition'] = test_tagtd[i].get_text(strip = True)
            
        if 'sallittu:' in test_tagth[i].get_text():
            apartment['allow_pets'] = test_tagtd[i].get_text(strip = True)
            
        if 'pinta-ala:' in test_tagth[i].get_text():
            apartment['total_size'] = test_tagtd[i].get_text(strip = True)
     
    try:
        # Push the collected data to db.
        table.insert({
                'postal_code': apartment['postal_code'],
                'built_year': apartment['built_year'], 
                'type': apartment['type'],
                'total_size': apartment['total_size'],
                'number_of_rooms': apartment['number_of_rooms'],
                'condition': apartment['condition'],
                'rent': apartment['rent'],
                'deposit': apartment['deposit'],
                'allow_pets': apartment['allow_pets']          
            }) 
        data_added += 1
    except pymongo.errors.DuplicateKeyError:
            pass
    
    
    
    #To check in mongodb on your laptop, install mongodb, type mongo to open terminal -
    # then 'use housing_data' -> db.etuovi.count() -- number of entries, db.etuovi.find() --check entries

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:49: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
